In [7]:
#!/usr/bin/env python
"""
OpenPose CSV → MMAction2 Skeleton PKL 변환 스크립트
하나의 PKL 파일에 전체 데이터와 90/10 split 정보(xsub_train/xsub_val)를 저장합니다.
balanced_true 폴더의 CSV는 label=1, false 폴더의 CSV는 label=0으로 설정합니다.
"""
from pathlib import Path
import pickle
import numpy as np
import pandas as pd
import random

# BODY_25 → COCO17 매핑
MAPPING_BODY25_TO_COCO17 = [
    0, 16, 15, 18, 17,
    5, 2, 6, 3, 7,
    4, 12, 9, 13, 10,
    14, 11
]


def load_and_process(csv_path: Path,
                     img_shape=(1080, 1920),
                     confidence_threshold=0.1,
                     interpolate=True,  # 기본 보간 활성화,
                     normalize_method='0to1') -> dict:
    """
    단일 CSV 파일을 읽어 annotation dict 반환
    """
    df = pd.read_csv(csv_path)
    T = len(df)
    V25 = 25

    kp25 = np.zeros((1, T, V25, 2), dtype=np.float32)
    score25 = np.zeros((1, T, V25), dtype=np.float32)
    for t, row in df.iterrows():
        vals = row.values.reshape(-1, 3)
        # NaN/Inf 처리: 이상치 0으로 대체
        vals = np.nan_to_num(vals, nan=0.0, posinf=0.0, neginf=0.0)
        kp25[0, t] = vals[:, :2]
        score25[0, t] = vals[:, 2]

    # confidence filter
    low = score25 < confidence_threshold
    kp25[low] = 0
    score25[low] = 0

    # interpolation
    h, w = img_shape
    if normalize_method == '0to1':
        kp25[..., 0] /= w
        kp25[..., 1] /= h
    elif normalize_method == 'center':
        kp25[..., 0] = (kp25[..., 0] - w/2) / (w/2)
        kp25[..., 1] = (kp25[..., 1] - h/2) / (h/2)
    elif normalize_method == 'skeleton_center':
        for t in range(T):
            pts = kp25[0, t]
            valid = np.all(pts != 0, axis=1)
            if valid.any():
                cxy = pts[valid].mean(axis=0)
                bbox = pts[valid]
                scale = max(bbox[:,0].ptp(), bbox[:,1].ptp())
                if scale > 0:
                    kp25[0, t] = (pts - cxy) / scale

    # COCO17 변환
    kp17 = kp25[:, :, MAPPING_BODY25_TO_COCO17, :]
    score17 = score25[:, :, MAPPING_BODY25_TO_COCO17]

    sample = {
        'total_frames': T,
        'img_shape': img_shape,
        'original_shape': img_shape,
        'keypoint': kp17,
        'keypoint_score': score17
    }
    return sample


if __name__ == '__main__':
    # 기본 경로 설정
    BASE = Path(r'D:/golfDataset/스포츠 사람 동작 영상(골프)/Training/Public/male/train')
    cats = ['balanced_true', 'false']

    # 전체 CSV 파일 목록 수집
    all_csvs = []
    for cat in cats:
        all_csvs.extend((BASE / cat / 'crop_keypoint').glob('*.csv'))
    all_csvs = sorted(all_csvs)

    # 90/10 split
    random.seed()   # 현재 시간 기반 시드 설정
    ids = [p.stem for p in all_csvs]
    random.shuffle(ids)
    split_idx = int(len(ids) * 0.9)
    train_set = set(ids[:split_idx])
    val_set = set(ids[split_idx:])

    # annotations 및 split 사전 초기화
    annotations = []
    split_dict = {'xsub_train': [], 'xsub_val': []}

    for csv_path in all_csvs:
        fid = csv_path.stem
        # annotation 생성
        info = load_and_process(csv_path, interpolate=True)
        # 폴더 이름으로 레이블 설정
        category = csv_path.parent.parent.name
        label = 1 if category == 'balanced_true' else 0
        info.update({'frame_dir': fid, 'label': label})
        annotations.append(info)

        # split 기록
        if fid in train_set:
            split_dict['xsub_train'].append(fid)
        else:
            split_dict['xsub_val'].append(fid)

    # 전체 데이터 패키징
    data = {'split': split_dict, 'annotations': annotations}

    # PKL 저장
    out_pkl = BASE / 'crop_pkl' / 'skeleton_dataset_90_10.pkl'
    out_pkl.parent.mkdir(parents=True, exist_ok=True)
    with open(out_pkl, 'wb') as f:
        pickle.dump(data, f, protocol=4)
    print(f"[DONE] Saved {out_pkl} with {len(annotations)} samples: "
          f"{len(split_dict['xsub_train'])} train, {len(split_dict['xsub_val'])} val")


[DONE] Saved D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\male\train\crop_pkl\skeleton_dataset_90_10.pkl with 436 samples: 392 train, 44 val
